## Calculating likelihoods

In [11]:
import matplotlib.pyplot as plt
import corner as corner
import pandas as pd
import numpy as np
import h5py
import torch
import time

from populations.bbh_models import get_models
import populations.bbh_models as read_models
from populations.utils.flow import NFlow
from populations.Flowsclass_dev import FlowModel
from populations import gw_obs

In [2]:
 #reading the pop synth file functions
def get_model_keys(path):
    alpha_val = '10'
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    f.close()

    # use only models with given alpha value
    for model in all_models:
        if 'alpha' in model:
            if 'alpha'+alpha_val in model:
                models.append('/'+model)
        else:
            models.append('/' + model)
    return(np.split(np.array(models), 5))

def get_model_keys_CE(path):
    all_models = []
    models = []
    def find_submodels(name ,obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    return(np.split(np.array(models), 4))

def read_hdf5(path, all_alpha=False):
    if all_alpha:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys_CE(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])
    else:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])

    return(popsynth_outputs)

In [27]:
params = ['mchirp','q', 'chieff', 'z']
file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
gw_path = '/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/gw_events'
observations, obsdata, p_theta, events = gw_obs.generate_observations(params, gw_path, \
                                            10000, 'posteriors', None)

popsynth_outputs = read_hdf5(file_path, True)
flow = FlowModel.from_samples('CE', popsynth_outputs, params, device='cpu')
flow.load_model('/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/flow_models/', 'CE')


TypeError: unhashable type: 'list'

In [19]:
observations, obsdata, p_theta, events = gw_obs.generate_observations(params, gw_path, \
                                            3, 'test', None)

In [6]:
print(np.shape(obsdata))

(46, 3, 4)


In [4]:
llks = flow(obsdata,[3,1])
llks

(46, 10000)
[-3.97861105e+00 -6.07484616e+00 -1.02101529e+01 -7.36015786e+00
 -1.32874797e-01 -6.16581793e+00 -2.62227205e+01 -1.48091157e-02
 -8.26205653e-02  1.00617512e+00 -2.48290837e+00 -2.62279036e+00
 -5.56160987e+00 -9.04078439e+00 -6.42973298e+00 -3.25261766e+01
 -7.29458759e+00 -4.28282795e+00 -9.65654473e+00 -2.88546126e+01
 -2.68162858e+01 -2.25253293e+01 -6.20223731e+00 -1.49162990e+01
 -3.00532040e+00 -6.81211561e+00 -1.46061702e+00 -7.26339036e-01
 -1.22298123e+00 -1.73047525e+00 -8.23899096e+00 -3.88733039e+00
 -7.31338903e+00 -2.53008956e+01 -2.42271698e+01 -6.96990828e+00
 -7.04269063e+00 -1.98085152e-01 -1.65881190e+01  0.00000000e+00
  4.55217520e-01 -4.31250048e+00 -5.12776458e+00 -1.25487474e+01
 -2.52573052e+01 -2.59583538e+01]


array([-3.97861105e+00, -6.07484616e+00, -1.02101529e+01, -7.36015786e+00,
       -1.32874797e-01, -6.16581793e+00, -2.62227205e+01, -1.48091157e-02,
       -8.26205653e-02,  1.00617512e+00, -2.48290837e+00, -2.62279036e+00,
       -5.56160987e+00, -9.04078439e+00, -6.42973298e+00, -3.25261766e+01,
       -7.29458759e+00, -4.28282795e+00, -9.65654473e+00, -2.88546126e+01,
       -2.68162858e+01, -2.25253293e+01, -6.20223731e+00, -1.49162990e+01,
       -3.00532040e+00, -6.81211561e+00, -1.46061702e+00, -7.26339036e-01,
       -1.22298123e+00, -1.73047525e+00, -8.23899096e+00, -3.88733039e+00,
       -7.31338903e+00, -2.53008956e+01, -2.42271698e+01, -6.96990828e+00,
       -7.04269063e+00, -1.98085152e-01, -1.65881190e+01,  0.00000000e+00,
        4.55217520e-01, -4.31250048e+00, -5.12776458e+00, -1.25487474e+01,
       -2.52573052e+01, -2.59583538e+01])

In [5]:
np.exp(llks)

array([1.87116108e-02, 2.30000000e-03, 3.67948417e-05, 6.36098040e-04,
       8.75574710e-01, 2.10000000e-03, 4.08900060e-12, 9.85300000e-01,
       9.20700427e-01, 2.73511949e+00, 8.35000234e-02, 7.26000000e-02,
       3.84258537e-03, 1.18477867e-04, 1.61288145e-03, 7.48274598e-15,
       6.79204981e-04, 1.38035711e-02, 6.40052958e-05, 2.94171752e-13,
       2.25857789e-12, 1.64958154e-10, 2.02489525e-03, 3.32608741e-07,
       4.95228846e-02, 1.10036250e-03, 2.32093025e-01, 4.83676475e-01,
       2.94351330e-01, 1.77200176e-01, 2.64150654e-04, 2.05000000e-02,
       6.66554246e-04, 1.02792313e-11, 3.00797251e-11, 9.39739091e-04,
       8.73772395e-04, 8.20300000e-01, 6.24987696e-08, 1.00000000e+00,
       1.57651627e+00, 1.34000013e-02, 5.92980129e-03, 3.54934510e-06,
       1.07372165e-11, 5.32635585e-12])